In [ ]:
from dataloader.result_loader import KinectResultLoader, ArbeResultLoader, OptitrackResultLoader
import numpy as np
import os
root_path = "/home/nesc525/chen/3DSVC/__test__/default"

arbe_loader = ArbeResultLoader(root_path)
arbe_loader.file_dict['arbe'] = arbe_loader.file_dict['arbe'].drop_duplicates(subset=['dt'],keep='first')
ids = arbe_loader.file_dict['arbe']['id']
ids = sorted(map(int, ids))
print(ids)
arbe_loader[ids[0]]["arbe"]
os.listdir(root_path)
frames = range(0, 5, 1)
print(frames)
ids[frames]

In [ ]:
import numpy as np
root_path = '/home/nesc525/drivers/4/p4tmesh/large4/6dwithgender/loss/corridor_test'
j_loss = np.load(os.path.join(root_path, 'joints_loss.npy'))
v_loss = np.load(os.path.join(root_path, 'vertices_loss.npy'))
print((j_loss[0]+j_loss[1])/2)
print((j_loss[-2]+j_loss[-1])/2)
print((v_loss[0]+v_loss[1])/2)
print((v_loss[-2]+v_loss[-1])/2)



In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from message.dingtalk import TimerBot

bot =TimerBot()
fig = plt.figure()
fig.add_subplot(1, 1, 1).plot([1,2,3], [1,2,3])
fig.canvas.draw()
img = cv2.cvtColor(np.asarray(fig.canvas.buffer_rgba()), cv2.COLOR_RGBA2BGR)
bot.add_md("test", "【TEST】 \n ![img]({})".format(bot.img2b64(img)))
bot.enable()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
fig.add_subplot(2, 2, 1, title="111",ylim=(0,200)).plot([1,2,3], [1,2,3])
fig.add_subplot(2, 2, 2, title="222",ylim=(0,200)).plot([1,2,3], [1,2,3])
fig.add_subplot(2, 2, 3, title="333",ylim=(0,200)).plot([1,2,3], [1,2,3])
fig.add_subplot(2, 2, 4, title="444",ylim=(0,200)).plot([1,2,3], [1,2,3])
fig.tight_layout()
plt.show()

In [ ]:
import torch
q=q.view(batch_size*length_size, 9, 3, 3)
q_blank=blank_atom.repeat(batch_size*length_size, 1, 1, 1)
pose=torch.cat((q_blank,
                q[:,1:3,:,:],
                q_blank,
                q[:,3:5,:,:],
                q_blank.repeat(1,10,1,1),
                q[:,5:9,:,:],
                q_blank.repeat(1,4,1,1)), 1)
rotmat=q[:,0,:,:]

In [ ]:
import torch
import torch.nn as nn
def rot_mat_2_euler(R):
    batch = R.size()[0]
    sy = torch.sqrt(R[:,0,0]*R[:,0,0]+R[:,1,0]*R[:,1,0])
    singular= sy<1e-6
    singular=singular.float()
        
    x=torch.atan2(R[:,2,1], R[:,2,2])
    y=torch.atan2(-R[:,2,0], sy)
    z=torch.atan2(R[:,1,0],R[:,0,0])
    
    xs=torch.atan2(-R[:,1,2], R[:,1,1])
    ys=torch.atan2(-R[:,2,0], sy)
    zs=R[:,1,0]*0
        
    out_euler=torch.autograd.Variable(torch.zeros(batch,3).cuda())
    out_euler[:,0]=x*(1-singular)+xs*singular
    out_euler[:,1]=y*(1-singular)+ys*singular
    out_euler[:,2]=z*(1-singular)+zs*singular
    
    return out_euler

def euler_2_rot_mat(euler):

    batch=euler.shape[0]
        
    c1=torch.cos(euler[:,0]).view(batch,1)#batch*1 
    s1=torch.sin(euler[:,0]).view(batch,1)#batch*1 
    c2=torch.cos(euler[:,2]).view(batch,1)#batch*1 
    s2=torch.sin(euler[:,2]).view(batch,1)#batch*1 
    c3=torch.cos(euler[:,1]).view(batch,1)#batch*1 
    s3=torch.sin(euler[:,1]).view(batch,1)#batch*1 
        
    row1=torch.cat((c2*c3,          -s2,    c2*s3         ), 1).view(-1,1,3) #batch*1*3
    row2=torch.cat((c1*s2*c3+s1*s3, c1*c2,  c1*s2*s3-s1*c3), 1).view(-1,1,3) #batch*1*3
    row3=torch.cat((s1*s2*c3-c1*s3, s1*c2,  s1*s2*s3+c1*c3), 1).view(-1,1,3) #batch*1*3
        
    matrix = torch.cat((row1, row2, row3), 1) #batch*3*3
     
    return matrix

def rotation6d_2_euler(nn_output):
    batch_size = nn_output.size()[0]
    num_joints = 9
    blank_atom=torch.tensor([[1,0,0],[0,1,0],[0,0,1]], dtype=torch.float32, requires_grad=False, device=torch.device('cuda:0'))
    q_blank=blank_atom.repeat(batch_size, 1, 1, 1)
    pose = nn_output[:,3:num_joints*6+3].reshape(batch_size*num_joints, 6).contiguous()
    tmp_x = nn.functional.normalize(pose[:,:3], dim = -1)
    tmp_z = nn.functional.normalize(torch.cross(tmp_x, pose[:,3:], dim = -1), dim = -1)
    tmp_y = torch.cross(tmp_z, tmp_x, dim = -1)

    tmp_x = tmp_x.view(batch_size,num_joints, 3, 1)
    tmp_y = tmp_y.view(batch_size,num_joints, 3, 1)
    tmp_z = tmp_z.view(batch_size,num_joints, 3, 1)
    pose = torch.cat((tmp_x, tmp_y, tmp_z), -1)
    R=torch.cat((q_blank,
                pose[:,1:3,:,:],
                q_blank,
                pose[:,3:5,:,:],
                q_blank.repeat(1,10,1,1),
                pose[:,5:9,:,:],
                q_blank.repeat(1,4,1,1)), 1).view(batch_size*24,3,3)
    rotmat=pose[:,0,:,:]

    R = torch.rand(24, 3, 3)

    euler = rot_mat_2_euler(R)
    mat = rot_mat_2_euler(euler)
    print(mat == R)
    euler = euler.view(batch_size, -1)

In [ ]:
R = torch.rand(24, 3, 3)

euler = rot_mat_2_euler(R)
mat = rot_mat_2_euler(euler)
print(mat == R)
euler = euler.view(batch_size, -1)

In [ ]:
import os
import cv2
import numpy as np
import torch
T = np.zeros((1,3), np.float32)
a = (0.2,0.4,0.8)
print ("a",a)
R = cv2.Rodrigues(a)
print ("R[0]", R[0])
v3 = (R[0][2,1],R[0][0,2],R[0][1,0])
print ("v3",v3)
c = cv2.Rodrigues(v3)
print ("c[0]",c[0])
b = cv2.Rodrigues(R[0])
print ("b[0]",b[0])
p = (-2.100418,-2.167796,0.27330)
print(cv2.Rodrigues(p)[0])

In [ ]:
import os

root_path = "/home/nesc525/drivers/2"
target_path = "/media/nesc525/perple/RGBD_data"
path = [p for p in os.listdir(root_path) if p[-1]=='T']
for p in path:
    os.system("cp -r {}/{}/rgbd_data {}/{}".format(root_path,p,target_path,p))

In [ ]:
import os
import numpy as np
path = "/home/nesc525/drivers/4/p4tmesh/new_test/loss"
# path = "/home/nesc525/drivers/4/p4tdepth/RGBD/loss"
total_j = 0
total_v = 0
len_j = 0
for p in os.listdir(path):
    try:
        j_loss = np.mean(np.load(os.path.join(path, p, 'per_joint_err.npy')), axis=1)
        v_loss = np.mean(np.load(os.path.join(path, p, 'per_vetex_err.npy')),axis=1)
    except Exception as e:
        # print(e)
        continue
    if p == 'test':
        continue
    print("s:", p)

    # print("max_j:", np.max(j_loss))
    # print("max_v:", np.max(v_loss))
    # print("mean_j:",np.mean(j_loss))
    # print("mean_v:",np.mean(v_loss))
    # print("var_j:",np.var(j_loss))
    print("var_v:",np.var(v_loss)*100)
    # print("len:",len(j_loss))
    total_j += np.sum(j_loss)
    total_v += np.sum(v_loss)
    len_j += len(j_loss)
# print(total_j/len_j)
# print(total_v/len_j)

In [ ]:
import os
path = "/home/nesc525/drivers/4/p4tdepth/depth"
# path = '/home/nesc525/drivers/4/p4tmesh/new_test'
for p in os.listdir(path):
    for t in os.listdir(path+"/loss"):
        if p == t[0]:
            os.system("cp -r {}/{}/loss/test/. {}/loss/{}".format(path,p,path,t))
            # os.system("rm -r {}/{}".format(path,p))

In [ ]:
import numpy as np

value = "0.228/0.249&	0.213/0.214&	0.162/0.182&	0.200/0.220&	0.202/0.223&	0.246/0.275&	0.243/0.265&	0.072/0.075"
value = list(map(lambda x: x.split('/'), value.split('&')))
value = np.asarray(value, np.float32)
for v in value:
    print(v-value[-1])

In [ ]:
import torch
import torch.nn.functional as F

t1 = torch.tensor([[1,2,3],[4,6,7]],dtype=float)
t2 = torch.tensor([[2,3,4],[5,5,6]],dtype=float)
print(F.mse_loss(t1, t2))
print(torch.norm(t1-t2, dim=1))
print(torch.mean(torch.norm(t1-t2, dim=1)))

In [ ]:
import numpy as np

s = np.load('/home/nesc525/drivers/1/2021-10-17_15-01-24_E/optitrack/id=1_st=1634454156.566333.npz')
print(s['markers'][0].shape)

In [ ]:
with open(os.path.join('/home/nesc525/drivers/1/2021-10-17_15-01-24_E/offsets.txt'), "r") as f:
    offset_dict = eval(f.readline())
    base = f.readline()
offset_dict['optitrack']
# base

In [ ]:
import numpy as np

s = np.load('/home/nesc525/drivers/2/2022-03-25_14-35-54/optitrack/id=3139_st=1648190232.866333.npz')
print(dict(s).keys())
print(s['markers'].shape)

In [ ]:
import cv2
from calib.utils import to_radar_transform_mat
trans = to_radar_transform_mat('/home/nesc525/drivers/1/2021-10-18_10-10-23_E')
rot_mat = cv2.Rodrigues(s['pose'][3:6])
print(cv2.Rodrigues(rot_mat[0])[0].shape)
a = np.array([2,5,6])
print(a.shape)
print(a @ rot_mat[0].T)
print(rot_mat[0] @ a)
print(a @ rot_mat[0].T + trans['optitrack']['t'])
print(rot_mat[0] @ a + trans['optitrack']['t'])

In [ ]:
import pickle
import time

path = '/home/nesc525/drivers/1/2021-10-18_18-21-24_E/pkl'
a = []
f_list = os.listdir(path)
f_list.sort(key=lambda x: int(x[3:-4]))
for p in f_list:
    t_s = time.time()

    with open(os.path.join(path, p), 'rb') as f:
        data = pickle.load(f)
    a.append(data)
    t_e = time.time()
    print(t_e - t_s)
a[1]['arbe'].shape

In [ ]:
import pickle
with open('/home/nesc525/drivers/1/2021-10-23_21-03-26_M/pkl/id=144.pkl', 'rb') as f:
    data = pickle.load(f)
    print(data['arbe'])
data.keys()


In [ ]:
from nn.p4t.datasets.folder_list import *
import os
path = []
driver_paths = ['/home/nesc525/drivers/1','/home/nesc525/drivers/2','/home/nesc525/drivers/3']
for pa in TRAIN_DIRS:
    path.append(pa)
for v in SELECTED_DIRS.values():
    path.append(v)
print(path)
for driver_path in driver_paths:
    for p in os.listdir(driver_path):
        if p not in path and os.path.exists(os.path.join(driver_path,p,'pkl_data/data.pkl')):
            print(driver_path, p)


In [ ]:
import os
import numpy as np
from dataloader.result_loader import ResultFileLoader
from minimal.bridge import JointsBridge
from visualization.utils import o3d_plot, o3d_pcl

driver_paths = ['/home/nesc525/drivers/1','/home/nesc525/drivers/2','/home/nesc525/drivers/3']
folders = ['2022-03-30_22-39-49_M']
for driver_path in driver_paths:
    for p in os.listdir(driver_path):
        if p in folders:
            root_path = os.path.join(driver_path, p)
res_loader = ResultFileLoader(root_path=root_path, enabled_sources=["arbe","master","sub2","kinect_skeleton","kinect_pcl","kinect_pcl_remove_zeros", "optitrack","mesh","mosh","mesh_param"])
for frame, _ in res_loader:
    bridge = JointsBridge()
    bridge.init_input(frame['master_skeleton'], frame['master_pcl'])
    k_jnts, pcl = bridge.map("kinect")
    joint_error = np.linalg.norm((k_jnts[:22] - frame['mesh_param']['joints'][:22]))
    o3d_plot([o3d_pcl(frame['master_pcl'], [0,1,0]), o3d_pcl(frame['master_skeleton'], [0,0,0]), o3d_pcl(k_jnts[:22], [1,0,0]), o3d_pcl(frame['mesh_param']['joints'][:22], [0,0,1])])
    print(joint_error)

In [ ]:
import os
import shutil

for test in ['lab1','lab2','furnished','rain','poor_lighting',]: #'smoke',
    f_name = os.path.join('/home/nesc525/drivers/4/p4t_mosh/mmWave/female', test, 'shape_err.npy')
    target_path = os.path.join('/home/nesc525/drivers/4/p4t_mosh/mmWave/shape_err', test)
    t_list = os.listdir(target_path)
    for p in t_list:
        if 'F' in p:
            shutil.copyfile(f_name, os.path.join(target_path, p))

In [ ]:
from visualization.utils import o3d_mesh
from mosh.utils import mosh_param_parser
import numpy as np

param = np.load('/home/nesc525/drivers/2/2022-03-29_11-50-15_F/mosh/param/id=0_st=1648525797.838.npz')
param
# mosh_param_parser

In [ ]:
from nn.p4t.datasets.folder_list import SELECTED_DIRS
import pickle
from sklearn.metrics import mean_squared_error
import numpy as np

beta = np.load('ignoredata/shape.npy')
average = np.mean(beta, 0)
num_frame = beta.shape[0]
print(beta.shape)
error = np.mean(np.square(np.repeat(average, num_frame).reshape(-1, 16) - beta), 0)
# err = np.linalg.norm((np.repeat(average, num_frame).reshape(-1, 16) - beta), axis=0)
# print(err**2/num_frame)
print(error)

In [ ]:
# np.repeat(average, num_frame, 0).shape
# beta.shape
print(beta[:2])
print(average)
print(np.linalg.norm((np.repeat(average, 2).reshape(-1, 16), beta[:2]), axis=1))

In [ ]:
import pickle
import time
from nn.p4t.datasets.mm_dataset import Depth

dataset = Depth('/home/nesc525/drivers/2', train=False, use_pkl=False, create_pkl=False)
t1 = time.time()
for i in range(5):
    # dataset[1]
    with open('/home/nesc525/drivers/1/2021-10-18_09-48-31_M/pkl/id=0_st=1634521727.9382143_dt=1634521727.938222.pkl', 'rb') as f:
    # with open('/home/nesc525/drivers/3/2022-03-30_20-49-18_M/pkl/id=3_st=1648644691.1595914_dt=1648644691.0662491.pkl', 'rb') as f:
        data = pickle.load(f)
    a = data[0].keys()
    print(a)
print(time.time() - t1)

In [ ]:
import torch
a = torch.Tensor([[[1,2],[3,4],[3,4]], [[7,8],[0,9],[3,4]]])
a.shape

In [ ]:
b = torch.Tensor([[1,2],[3,4]])
b = b.unsqueeze(-2).repeat(1,3,1)
b.shape

In [ ]:
import pickle
import time

for i in range(5):
    ts = time.time()
    with open('/home/nesc525/drivers/3/2022-03-30_20-49-18_M/pkl/id=8_st=1648644691.2760816_dt=1648644691.2832491.pkl', 'rb') as f:
        data = pickle.load(f)
    print(time.time() - ts)

In [ ]:
import torch

a = torch.zeros([2,3,1024,2048])
a[0,:,255,1023] = torch.ones_like(a[0,:,255,1023])
a[0,:,1023,2047] = torch.ones_like(a[0,:,256,1024])
print(a[0,:,255,1023])
b = torch.ones([2,1024,1,2],dtype=torch.float32)
b[0,0,0,:] = torch.tensor([-0.0009765625, -0.5014662756598240469208211143695])
c = torch.nn.functional.grid_sample(a, b)
print(c.permute(3,0,2,1))

In [ ]:
import zipfile
 
z = zipfile.ZipFile("/home/nesc525/drivers/0/chen/dataset.zip", "r")

files = []
for filename in z.namelist():
    if filename.endswith('md'):
        files.append(filename)
        print('File:', filename)
content = z.read(files[-2])
print(content)

In [ ]:
a = '[1,2,3, 4]'
b = eval(a)
print(b)

In [ ]:
import torch.nn as nn
import torch

l = nn.Conv1d(512, 1024, (1, 1))
a = torch.ones(32,512,7,7)
b = l(a)
b.shape

In [ ]:
import numpy as np
a = np.array([1.0])
# np.float32(a)
a.astype(float)

In [ ]:
from sync.offsets import CalibOffsets

root_path = ''
CalibOffsets.from_file(root_path)

In [ ]:
import os
from dataloader.result_loader import ResultFileLoader

enabled_sources = 'arbe,optitrack,sub2,kinect_pcl,kinect_pcl_remove_zeros,mesh,mosh,mesh_obj'.split(',')
path = '/home/nesc525/drivers/2/2021-10-20_14-08-40_F'
skip_head = 0
skip_tail = 0
data_loader = ResultFileLoader(root_path=path, skip_head=skip_head, skip_tail=skip_tail, enabled_sources=enabled_sources)
for i in range(len(data_loader)):
    frame, _ = data_loader[i]
    pass

In [1]:
import numpy as np
import torch
from nn.datasets.utils import project_pcl_torch

pcl = np.array([[[1,2,3], [4,5,6]]])
project_pcl_torch(torch.from_numpy(pcl).cuda())

tensor([[[1427, 1348],
         [1588, 1671]]], device='cuda:0', dtype=torch.int32)

In [2]:
h = [2048] * (3 - 1)

[(n, k) for n, k in zip([1024] + h, h + [1024])]

[(1024, 2048), (2048, 2048), (2048, 1024)]